<a href="https://colab.research.google.com/github/Rumata-arc/Probabilities_Surprisal/blob/main/Token_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#БЛОК 1. Импорт библиотек и базовые настройки#

import os
import pandas as pd
import numpy as np


In [3]:
#БЛОК 2. Загрузка и подготовка текстов из data_exp.xlsx#

from google.colab import files
import pandas as pd
import os

# === ЗАГРУЗКА ФАЙЛА ЧЕРЕЗ COLAB ===
print("Загрузите файл data_exp.xlsx")
uploaded = files.upload()

# проверка
if "data_exp.xlsx" not in uploaded:
    raise FileNotFoundError(
        "Файл data_exp.xlsx не загружен. Пожалуйста, загрузите именно его."
    )

DATA_EXP = "data_exp.xlsx"

# === ЗАГРУЗКА И ПОДГОТОВКА ТЕКСТОВ ===
GLOBAL_TEXTS = {}

xls = pd.ExcelFile(DATA_EXP)

for sheet in xls.sheet_names:
    if not sheet.startswith("Text_"):
        continue

    df = pd.read_excel(DATA_EXP, sheet_name=sheet)

    words = []
    for col in df.columns:
        for cell in df[col].dropna():
            w = str(cell).strip()
            if w != "":
                words.append(w)

    text = " ".join(words)

    GLOBAL_TEXTS[sheet] = {
        "text": text,
        "words_count": len(words),
        "chars_count": len(text)
    }

print(f"Загружено текстов: {len(GLOBAL_TEXTS)}")




Загрузите файл data_exp.xlsx


Saving data_exp.xlsx to data_exp.xlsx
Загружено текстов: 30


In [4]:
#БЛОК 3. Расчёт токенов, вероятностей и surprisal (Qwen)#

def process_single_text_correct(text, text_id, max_tokens=1024):
    """
    Возвращает список словарей:
    position, token, token_id, probability, log_probability,
    surprisal_ln, surprisal_log2
    """

    results = []

    # !!! здесь используется твой код !!!
    # tokenizer, model, torch, device — всё как у тебя

    # пример структуры результата:
    # results.append({
    #     "position": pos,
    #     "token": token_str,
    #     "token_id": token_id,
    #     "probability": prob,
    #     "log_probability": log_prob,
    #     "surprisal_ln": -log_prob,
    #     "surprisal_log2": -np.log2(prob)
    # })

    return results



In [12]:
# === БЛОК 4 (COLAB): загрузка zip с probabilities.csv + word-level агрегация ===
# Статьи и тезисы/псхология XXI век 2026/ в секцию психофизиологии
# Исправления:
# 1) Авто-находит вложенную папку с Text_1_probabilities.csv
# 2) NaN в token -> '' (чтобы не появлялось "nan")
# 3) token_id -1 и 220: участвуют в сборке слова (acc), но НЕ в метриках (sum/mean)
# 4) Нормализация тире/минусов: '–','—','−',... -> '-' (и для word, и для token)
# 5) Спец-правило регистра для самого первого символа (н vs Н) в token_id=-1

import os
import zipfile
import pandas as pd
import numpy as np
from google.colab import files

DATA_EXP = "data_exp.xlsx"
OUT_DIR = "WORD_LEVEL_RESULTS"

SKIP_FOR_METRICS = {-1, 220}  # не включаем в used, но можем использовать в сборке слова

os.makedirs(OUT_DIR, exist_ok=True)

def collect_words(sheet_df):
    words = []
    for col in sheet_df.columns:
        for cell in sheet_df[col].dropna():
            w = str(cell).strip()
            if w != "":
                words.append(w)
    return words

def find_probs_root(extract_root):
    """Ищет папку, где лежат Text_1_probabilities.csv и т.п."""
    if os.path.exists(os.path.join(extract_root, "Text_1_probabilities.csv")):
        return extract_root

    entries = [e for e in os.listdir(extract_root) if os.path.isdir(os.path.join(extract_root, e))]
    if len(entries) == 1:
        candidate = os.path.join(extract_root, entries[0])
        if os.path.exists(os.path.join(candidate, "Text_1_probabilities.csv")):
            return candidate

    for root, _, files_ in os.walk(extract_root):
        if "Text_1_probabilities.csv" in files_:
            return root

    return None

def safe_tok_str(x):
    """NaN/None -> '' ; иначе строка."""
    if x is None:
        return ""
    try:
        if pd.isna(x):
            return ""
    except Exception:
        pass
    return str(x)

def norm_text(s: str) -> str:
    """Нормализуем разные тире/минусы к обычному дефису '-'."""
    if s is None:
        return ""
    return (s.replace("–", "-")   # en dash
             .replace("—", "-")   # em dash
             .replace("−", "-")   # minus
             .replace("‒", "-")   # figure dash
             .replace("-", "-")   # non-breaking hyphen
             .replace("﹘", "-")
             .replace("－", "-"))

# === 1) UPLOAD ZIP ===
print("Загрузите архив с probabilities (zip)")
uploaded = files.upload()

zip_name = None
for k in uploaded.keys():
    if k.lower().endswith(".zip"):
        zip_name = k
        break
if zip_name is None:
    raise FileNotFoundError("Не найден .zip среди загруженных файлов.")

# === 2) EXTRACT ===
EXTRACT_ROOT = "Texts_probabilities_FINAL_csv"
os.makedirs(EXTRACT_ROOT, exist_ok=True)

with zipfile.ZipFile(zip_name, "r") as z:
    z.extractall(EXTRACT_ROOT)

# === 3) FIND REAL DIR WITH CSVs ===
QWEN_DIR = find_probs_root(EXTRACT_ROOT)
if QWEN_DIR is None:
    print("Содержимое корня распаковки:", os.listdir(EXTRACT_ROOT))
    raise FileNotFoundError("Не удалось найти папку с Text_1_probabilities.csv внутри распакованного архива.")

print("✅ Папка с CSV:", QWEN_DIR)

# === 4) WORD-LEVEL AGGREGATION ===
for text_id in range(1, 31):
    print(f"Processing Text_{text_id}")

    # --- слова из Excel ---
    words_df = pd.read_excel(DATA_EXP, sheet_name=f"Text_{text_id}")
    words = collect_words(words_df)

    # --- токены Qwen ---
    tok_path = os.path.join(QWEN_DIR, f"Text_{text_id}_probabilities.csv")
    tok_df = pd.read_csv(tok_path).sort_values("position").reset_index(drop=True)

    results = []
    tok_idx = 0

    for word in words:
        word_raw = str(word)
        word_norm = norm_text(word_raw)

        acc_raw = ""
        acc_norm = ""
        used = []

        while tok_idx < len(tok_df) and acc_norm != word_norm:
            t = tok_df.iloc[tok_idx]

            tid = t.get("token_id", None)
            try:
                tid = int(tid)
            except Exception:
                tid = None

            tok_raw = safe_tok_str(t.get("token", ""))
            tok_norm = norm_text(tok_raw)

            # Служебные токены: НЕ в метрики, но участвуют в сборке слова, если не пустые
            if tid in SKIP_FOR_METRICS:
                if tok_raw.strip() != "":
                    # спец-правило: если первый символ отличается только регистром (н vs Н)
                    if tid == -1 and acc_norm == "" and len(tok_raw) == 1 and len(word_raw) > 0:
                        if tok_raw.casefold() == word_raw[0].casefold() and tok_raw != word_raw[0]:
                            tok_raw = word_raw[0]
                            tok_norm = norm_text(tok_raw)

                    acc_raw += tok_raw
                    acc_norm += tok_norm

                tok_idx += 1
                continue

            # Обычные токены: и в сборку, и в метрики
            acc_raw += tok_raw
            acc_norm += tok_norm
            used.append(t)
            tok_idx += 1

        if acc_norm != word_norm:
            nxt = [safe_tok_str(x) for x in tok_df.iloc[tok_idx:tok_idx+30]["token"].tolist()]
            preview = "".join(nxt)[:200]
            raise ValueError(
                f"Не удалось собрать слово '{word_raw}' в Text_{text_id}, получено '{acc_raw}'. "
                f"Следующие токены дают: '{preview}'"
            )

        df_used = pd.DataFrame(used)

        # Если слово собрано только из служебных токенов (редко), метрики будут 0
        if len(df_used) == 0:
            sum_ln = mean_ln = sum_l2 = mean_l2 = 0.0
            start_pos = end_pos = None
        else:
            sum_ln = df_used["surprisal_ln"].sum()
            mean_ln = df_used["surprisal_ln"].mean()
            sum_l2 = df_used["surprisal_log2"].sum()
            mean_l2 = df_used["surprisal_log2"].mean()
            start_pos = df_used["position"].iloc[0]
            end_pos = df_used["position"].iloc[-1]

        results.append({
            "text_id": f"Text_{text_id}",
            "word": word_raw,
            "n_tokens": int(len(df_used)),
            "sum_surprisal_ln": float(sum_ln),
            "mean_surprisal_ln": float(mean_ln),
            "sum_surprisal_log2": float(sum_l2),
            "mean_surprisal_log2": float(mean_l2),
            "start_position": start_pos,
            "end_position": end_pos,
        })

    out_df = pd.DataFrame(results)
    out_path = os.path.join(OUT_DIR, f"Text_{text_id}_word_surprisal.csv")
    out_df.to_csv(out_path, index=False, encoding="utf-8-sig")

print("ГОТОВО. Результаты в папке:", OUT_DIR)









Загрузите архив с probabilities (zip)


Saving Probabilities csv.zip to Probabilities csv (1).zip
✅ Папка с CSV: Texts_probabilities_FINAL_csv/Probabilities csv
Processing Text_1
Processing Text_2
Processing Text_3
Processing Text_4
Processing Text_5
Processing Text_6
Processing Text_7
Processing Text_8
Processing Text_9
Processing Text_10
Processing Text_11
Processing Text_12
Processing Text_13
Processing Text_14
Processing Text_15
Processing Text_16
Processing Text_17
Processing Text_18
Processing Text_19
Processing Text_20
Processing Text_21
Processing Text_22
Processing Text_23
Processing Text_24
Processing Text_25
Processing Text_26
Processing Text_27
Processing Text_28
Processing Text_29
Processing Text_30
ГОТОВО. Результаты в папке: WORD_LEVEL_RESULTS


In [11]:
# =========================
# БЛОК 5: Собрать все Text_*_word_surprisal.csv в один Excel и скачать
# =========================
import os
import glob
import pandas as pd
from google.colab import files

OUT_DIR = "WORD_LEVEL_RESULTS"  # как выше
xlsx_path = os.path.join(OUT_DIR, "ALL_TEXTS_word_surprisal.xlsx")

csv_files = sorted(glob.glob(os.path.join(OUT_DIR, "Text_*_word_surprisal.csv")))
if not csv_files:
    raise FileNotFoundError(f"В папке {OUT_DIR} не найдено файлов Text_*_word_surprisal.csv")

# Соберём сводку по текстам
summary_rows = []
per_text_dfs = []

for fp in csv_files:
    df = pd.read_csv(fp)
    text_id = df["text_id"].iloc[0] if "text_id" in df.columns and len(df) else os.path.basename(fp).split("_word_surprisal.csv")[0]

    # базовые агрегаты по словам (исключим "служебные" слова с n_tokens=0, если они есть)
    df_nonzero = df[df.get("n_tokens", 1) > 0] if "n_tokens" in df.columns else df

    summary_rows.append({
        "text_id": text_id,
        "words_total": int(len(df)),
        "words_nonzero_tokens": int(len(df_nonzero)),
        "mean_mean_surprisal_ln": float(df_nonzero["mean_surprisal_ln"].mean()) if len(df_nonzero) else None,
        "mean_sum_surprisal_ln": float(df_nonzero["sum_surprisal_ln"].mean()) if len(df_nonzero) else None,
        "mean_mean_surprisal_log2": float(df_nonzero["mean_surprisal_log2"].mean()) if len(df_nonzero) else None,
        "mean_sum_surprisal_log2": float(df_nonzero["sum_surprisal_log2"].mean()) if len(df_nonzero) else None,
    })
    per_text_dfs.append((text_id, df))

summary_df = pd.DataFrame(summary_rows)

# Запишем в Excel: 1) summary 2) каждый Text_i отдельным листом
with pd.ExcelWriter(xlsx_path, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="TEXT_SUMMARY", index=False)

    for text_id, df in per_text_dfs:
        # имя листа <= 31 символа
        sheet_name = str(text_id)[:31]
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Excel создан:", xlsx_path)
print("📦 Размер (байт):", os.path.getsize(xlsx_path))

# Скачать
files.download(xlsx_path)



✅ Excel создан: WORD_LEVEL_RESULTS/ALL_TEXTS_word_surprisal.xlsx
📦 Размер (байт): 377489


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>